# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,236.698838,-0.972740,1234,235.726098,237.671579,471.452197,475.343157
6,543.549331,0.448333,1234,543.997664,543.100997,1087.995328,1086.201994
7,328.160735,0.616593,1234,328.777328,327.544142,657.554656,655.088285
8,749.247283,-0.231082,1234,749.016201,749.478366,1498.032402,1498.956731
9,76.643544,-0.688043,1234,75.955501,77.331586,151.911002,154.663172
10,691.119807,0.710755,1234,691.830562,690.409052,1383.661124,1380.818104
11,119.791137,-0.349234,1234,119.441903,120.140371,238.883807,240.280742
12,635.764695,-0.773867,1234,634.990828,636.538561,1269.981656,1273.077123
13,269.575668,-0.683217,1234,268.892451,270.258886,537.784902,540.517772
14,593.081876,0.605566,1234,593.687442,592.476310,1187.374884,1184.952620


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-235.726098,237.671579
6,-543.997664,543.100997
7,-328.777328,327.544142
8,-749.016201,749.478366
9,-75.955501,77.331586
10,-691.830562,690.409052
11,-119.441903,120.140371
12,-634.990828,636.538561
13,-268.892451,270.258886
14,-593.687442,592.476310


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-235.726098,237.671579
6,-543.997664,543.100997
7,-328.777328,327.544142
8,-749.016201,749.478366
9,-75.955501,77.331586
10,-691.830562,690.409052
11,-119.441903,120.140371
12,-634.990828,636.538561
13,-268.892451,270.258886
14,-593.687442,592.476310


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-235.72609838,  237.6715786 ],
       [-543.99766391,  543.10099712],
       [-328.77732784,  327.54414226],
       [-749.01620093,  749.47836567],
       [ -75.95550102,   77.33158606],
       [-691.83056195,  690.40905209],
       [-119.4419034 ,  120.14037085],
       [-634.99082821,  636.53856149],
       [-268.89245086,  270.25888583],
       [-593.6874418 ,  592.47631005]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,469.452197,-475.343157
6,1085.995328,-1086.201994
7,655.554656,-655.088285
8,1496.032402,-1498.956731
9,149.911002,-154.663172
10,1381.661124,-1380.818104
11,236.883807,-240.280742
12,1267.981656,-1273.077123
13,535.784902,-540.517772
14,1185.374884,-1184.952620


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
